### Bibliotecas

In [1]:
import pandas as pd
import os
import pickle
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata
import torch

C:\Users\Bodoque\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Modelos para extraer embeddings

In [2]:
MODELOS = {
    "sentence-transformers": 'Santp98/SBERT-pairs-bert-base-spanish-wwm-cased',
    "sentence_similarity": 'hiiamsid/sentence_similarity_spanish_es'
}

CAMPOS = ["Titulo", "resumen", "objetivos", "claves"]

### Extracción de NER's

In [6]:
def cargar_modelo_ner(nombre_modelo="iEsmeralda/mrm8488-finetuned-ner-tech"):
    tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)
    model = AutoModelForTokenClassification.from_pretrained(nombre_modelo)
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    return ner_pipeline

def agrupar_entidades_consecutivas(entidades):
    if not entidades:
        return []
    entidades_agrupadas = []
    entidad_actual = entidades[0].copy()
    for i in range(1, len(entidades)):
        entidad = entidades[i]
        if entidad["entity_group"] == entidad_actual["entity_group"] and entidad["start"] == entidad_actual["end"] + 1:
            entidad_actual["word"] += " " + entidad["word"]
            entidad_actual["end"] = entidad["end"]
            entidad_actual["score"] = (entidad_actual["score"] + entidad["score"]) / 2
        else:
            entidades_agrupadas.append(entidad_actual)
            entidad_actual = entidad.copy()
    entidades_agrupadas.append(entidad_actual)
    return entidades_agrupadas

def extraer_ners(texto, ner_pipeline, max_tokens=512):
    tokenizer = ner_pipeline.tokenizer
    model = ner_pipeline.model

    tokens = tokenizer.tokenize(texto)
    entidades = []

    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i+max_tokens]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)

        # se asegura de que no se pase del limite del modelo de sentence_transformer y sentence_similarity (512 tokens)
        if len(tokenizer(chunk_text)["input_ids"]) > max_tokens:
            continue

        resultado_chunk = ner_pipeline(chunk_text)
        entidades_chunk = agrupar_entidades_consecutivas(resultado_chunk)
        entidades.extend(entidades_chunk)

    return [entidad["word"] for entidad in entidades]


### Obtener embeddings

In [ ]:
def cargar_modelo(nombre_modelo):
    return SentenceTransformer(nombre_modelo)

def obtener_embeddings(df, campo, modelo, ner_pipeline, max_tokens=512):
    tokenizer = modelo.tokenizer if hasattr(modelo, 'tokenizer') else None
    textos = df[campo].astype(str).tolist()
    embeddings = []

    for texto in textos:
        ners = extraer_ners(texto, ner_pipeline)
        texto_enriquecido = f"{texto} {' '.join(ners)}"

        # tokeniza texto completo enriquecido
        tokens = tokenizer.tokenize(texto_enriquecido) if tokenizer else texto_enriquecido.split()

        if len(tokens) <= max_tokens:
            vector = modelo.encode(texto_enriquecido)
        else:
            chunk_embeddings = []
            for i in range(0, len(tokens), max_tokens):
                chunk_text = " ".join(tokens[i:i+max_tokens])
                vec = modelo.encode(chunk_text)
                chunk_embeddings.append(vec)
            vector = sum(chunk_embeddings) / len(chunk_embeddings)

        embeddings.append(vector)

    return embeddings

def guardar_embeddings(embeddings, archivo_salida):
    with open(archivo_salida, 'wb') as f:
        pickle.dump(embeddings, f)

In [3]:
protocolos=pd.read_csv("protocolos_completo_limpios.csv")

In [4]:
protocolos_acentuados=pd.read_csv("protocolos_completo_limpios.csv")

In [5]:
def normalizar_texto(texto):
    texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')
    return texto.lower()
protocolos = protocolos.applymap(lambda x: normalizar_texto(x) if isinstance(x, str) else x)

C:\Users\Bodoque\AppData\Local\Temp\ipykernel_20056\4239636668.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  protocolos = protocolos.applymap(lambda x: normalizar_texto(x) if isinstance(x, str) else x)


In [ ]:
def main():
    df = protocolos
    campo = input("Selecciona el campo (Titulo, resumen, objetivos, claves) para obtener sus PKL's: ").strip()

    if campo not in df.columns:
        print("El campo ingresado es invalido.")
        return

    ruta_destino = os.path.join("pkl")
    os.makedirs(ruta_destino, exist_ok=True)

    ner_pipeline = cargar_modelo_ner()

    for clave_modelo, nombre_modelo in MODELOS.items():
        print(f"\nProcesando modelo: {clave_modelo}")
        modelo = cargar_modelo(nombre_modelo)
        embeddings_obtenidos = obtener_embeddings(df, campo, modelo, ner_pipeline)
        archivo_embeddings = os.path.join(ruta_destino, f"{campo}_{clave_modelo}_embeddings.pkl")
        guardar_embeddings(embeddings_obtenidos, archivo_embeddings)

if __name__ == "__main__":
    main()

### Comparación de campos

In [7]:
df = protocolos
df_resultados_acentuados  = protocolos_acentuados
ner_pipeline = cargar_modelo_ner()

while True:
    query = input("\nIngresa el texto para comparar (o escribe 'salir' para terminar): ").strip()
    if query.lower() == "salir":
        break

    print(f"\nConsulta original: {query}")
    entidades_query = extraer_ners(query, ner_pipeline)
    print("Entidades NER detectadas:", entidades_query)

    query_enriquecida = f"{query} {' '.join(entidades_query)}" if entidades_query else query
    query_con_contexto = f"Este trabajo trata sobre {query_enriquecida}"

    df_resultados_acentuados = protocolos_acentuados.copy()
    df_resultados_acentuados["sim_total"] = 0

    for clave_modelo, nombre_modelo in MODELOS.items():
        modelo = SentenceTransformer(nombre_modelo)
        embedding_query = modelo.encode(query_con_contexto, convert_to_tensor=False)

        for campo in CAMPOS:
            nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
            ruta_pkl = os.path.join("pkl", nombre_pkl)

            if not os.path.exists(ruta_pkl):
                print(f"Falta el archivo: {ruta_pkl}")
                continue

            with open(ruta_pkl, "rb") as f:
                embeddings_cargados = pickle.load(f)

            simi_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
            df_resultados_acentuados[f"{campo}_{clave_modelo}"] = simi_coseno
            df_resultados_acentuados["sim_total"] += simi_coseno # ponderacion simple

    df_resultados_acentuados = df_resultados_acentuados.sort_values("sim_total", ascending=False).head(5)

    print("\nTop 5 resultados más similares:")
    for i, row in df_resultados_acentuados.iterrows():
        print(f"\nTT: {row['TT']}")
        for campo in CAMPOS:
            resumen = str(row[campo])
            if len(resumen) > 180:
                resumen = resumen[:180] + "..."
            print(f"- {campo.capitalize()}: {resumen}")
        print(f"Similitud total: {row['sim_total']:.4f}")

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Consulta original: redes neuronales convolucionales
Entidades NER detectadas: ['redes neuronales convolucionales']

Top 5 resultados más similares:

TT: 2016-B026
- Titulo: Reconocimiento del Tránsito vehicular aplicando Redes Neuronales Profundas
- Resumen: Tomando en cuenta el incremento del congestionamiento vial y el aumento de cámaras en  las calles en la Ciudad de México proponemos desarrollar una herramienta que realice el  reco...
- Objetivos: Objetivos General Implementar una Red Neuronal Convolucional que haga uso del video tomado por cámaras  para reconocer y clasificar el congestionamiento vehicular. Particulares  Im...
- Claves: Deep Learning, Redes Neuronales Profundas o Convolucionales, Visión por Computadora.
Similitud total: 4.6636

TT: 2016-B029
- Titulo: Sistema de Clasificación de Razas Caninas Mediante Redes Neuronales Convolucionales
- Resumen: En el presente protocolo se describe una propuesta para desarrollar un sistema computacional con arquitectura cliente- s